# "Berlin Net Mel Bn"

In [1]:
import keras
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram
from DataGenerator import DataGenerator

Using TensorFlow backend.


### Load Data

In [72]:
data_path='../preprocessing/preprocessed_data'
params = {'batch_size': 64,
          'shuffle': True}

In [73]:
training_generator = DataGenerator(data_path, ['train/voxforge', 'train/youtube'], num=80000, **params)
validation_generator = DataGenerator(data_path, ['val/youtube'], num=5000, **params)

### Define Model

Similar Architecture to [paper](https://github.com/twerkmeister/iLID/blob/master/Deep%20Audio%20Paper%20Thomas%20Werkmeister%2C%20Tom%20Herold.pdf)

In [79]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=28,
                         fmin=0.0, fmax=10000, power_melgram=1.0,
                         return_decibel_melgram=False, trainable_fb=False,
                         trainable_kernel=False))
model.add(layers.Conv2D(64, (3, 6), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 6), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 6), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [80]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_2 (Melspectro (None, 28, 313, 1)        270364    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 308, 64)       1216      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 154, 64)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 149, 64)       73792     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 74, 64)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 69, 128)        147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 1, 34, 128)        0         
__________

In [81]:
model.compile(optimizer='Rmsprop',
              metrics=['accuracy'],
              loss='categorical_crossentropy')

### Train Model

In [82]:
history = model.fit_generator(generator=training_generator,
                              validation_data=validation_generator,
                              verbose=1,
                              epochs=5,
                              use_multiprocessing=True,
                              workers=8)

Epoch 1/5
1250/1250 [==============================] - 372s 297ms/step - loss: 0.7179 - acc: 0.6837 - val_loss: 0.4510 - val_acc: 0.8217
Epoch 2/5
1250/1250 [==============================] - 314s 251ms/step - loss: 0.4242 - acc: 0.8369 - val_loss: 0.3349 - val_acc: 0.8726
Epoch 3/5
1250/1250 [==============================] - 319s 255ms/step - loss: 0.3332 - acc: 0.8758 - val_loss: 0.3320 - val_acc: 0.8744
Epoch 4/5
1250/1250 [==============================] - 303s 242ms/step - loss: 0.2860 - acc: 0.8954 - val_loss: 0.3232 - val_acc: 0.8760
Epoch 5/5
1250/1250 [==============================] - 317s 254ms/step - loss: 0.2605 - acc: 0.9059 - val_loss: 0.2975 - val_acc: 0.8888


### Save Model
89% Accuracy Test is not bad!!

In [85]:
model.save('berlin_net_mel_bn.h5')